# Exploratory Data Analysis (EDA) for Insurance Fraud Dataset

This notebook explores the cleaned **Insurance Claims** Dataset to understand:
- **Data Structure** and **Summary Statistics**
- `Fraud` vs `non-fraud` **Distribution** and **Imbalance**
- **Distribution** of key **Numeric Features**
- **Relationships** between **Features** and **Fraud**
- **Categorical** **Feature Analysis**
- **Time-based** **Trends** (if available)
- **Outlier** **Detection**

I use `pandas` for **Data Manipulation** and `seaborn`/`matplotlib` for **Visualization**.

## Table of Contents
1. Load Data and Basic Inspection  
2. Data Overview and Summary Statistics  
3. Fraud Class Distribution  
4. Convert Fraud Labels for Numeric Analysis  
5. Numeric Feature Exploration  
6. Categorical Feature Exploration  
7. Time-Based Trends 
8. EDA Summary and Next Steps

In [ ]:
# Importing essential Python libraries
import pandas as pd                                 # For data manipulation
import seaborn as sns                               # For statistical visualizations
import numpy as np                                  # For numerical operations
import matplotlib.pyplot as plt                     # For basic plotting
import os                                           # For file path handling
from IPython.display import display, Markdown       # For displaying Markdown and DataFrames in Jupyter

# Set Seaborn's visual style for cleaner plots
sns.set(style="whitegrid")

# Enable inline plotting for Jupyter
%matplotlib inline

## 1. Load Data and Basic Inspection

In this step, I load the cleaned insurance claims dataset from a CSV file saved from the ETL process into a pandas DataFrame. I also inspect the shape (rows, columns) to understand dataset size and display the first few rows to get an initial feel for the data’s structure and content.

In [ ]:
# Define project directory and cleaned data path
project_dir = r"C:\Users\Cloud\OneDrive\Desktop\Fraud_Analytics_Project"
cleaned_file = os.path.join(project_dir, "data", "cleaned", "cleaned_insurance_claims.csv")

# Load cleaned dataset with date parsing for incident_date and policy_bind_date
try:
    # Attempt to load the cleaned dataset, parsing dates in specified columns
    df = pd.read_csv(cleaned_file, parse_dates=['incident_date', 'policy_bind_date'])
    # This will inform me that the file was loaded successfully with the exact path.
    display(Markdown(f"**Loaded data from:** `{cleaned_file}`"))
except FileNotFoundError:
    # This line catches the specific error when the file is not found.
    display(Markdown(f"**Error:** File not found at `{cleaned_file}`. Please check the file path."))
    # Raise error or exit
    raise
except Exception as e:
    # Catches any other unexpected exceptions during file loading.
    display(Markdown(f"**Error loading data:** {e}"))
    # Raises the error again after displaying the message.
    raise

# Display dataframe shape
display(Markdown("**Dataframe shape:**"))
display(df.shape)

# Show first 5 rows for initial inspection
display(Markdown("**First 5 rows:**"))
display(df.head())

# Show dataframe info for datatypes and non-null counts
display(Markdown("**Dataframe info:**"))
df.info()

# Confirm 'fraud_reported' column is numeric and show value counts
display(Markdown("**Fraud reported value counts:**"))
display(df['fraud_reported'].value_counts())

# Optionally create a human-readable label for plotting convenience
df['fraud_label'] = df['fraud_reported'].map({0: 'No', 1: 'Yes'})

# Display sample of fraud_reported and fraud_label columns
display(Markdown("**Sample fraud_reported and fraud_label:**"))
display(df[['fraud_reported', 'fraud_label']].head())

In [ ]:
[col for col in df.columns if 'missing' in col.lower()]

## 2. Data Overview and Summary Statistics

Here, I review the dataset's structure using `info()` to check data types and missing values. I then display summary statistics (mean, median, quartiles, etc.) for numeric columns, helping me understand feature distributions and spot anomalies.

In [ ]:
# Count how many missing values in each column, sorted descending
missing_counts = df.isna().sum().sort_values(ascending=False)

# Calculate percentage of missing values relative to total rows, rounded to 2 decimals
missing_percent = (missing_counts / len(df) * 100).round(2)

# Combine missing counts and percentages into a DataFrame for easier viewing
missing_summary = pd.DataFrame({'Missing Count': missing_counts, 'Missing %': missing_percent})

# Display missing value summary table
display(missing_summary)

# List all columns that have any missing values
cols_with_missing = missing_counts[missing_counts > 0].index.tolist()
display(Markdown(f"**Columns with missing values:** {cols_with_missing}"))

## 3. Fraud Class Distribution

In this section, I explore the distribution of fraudulent vs non-fraudulent claims.
This is important because fraud datasets are often **imbalanced**, which can affect model performance. I show both the **counts and percentages** of each class, and visualize the imbalance using an annotated bar chart.

In [ ]:
# Count number of fraud (1) and non-fraud (0) cases
fraud_counts = df['fraud_reported'].value_counts().sort_index()
fraud_percentages = (fraud_counts / fraud_counts.sum() * 100).round(2)

# Plot countplot using numeric labels and annotated percentages
plt.figure(figsize=(6, 4))
ax = sns.countplot(x='fraud_reported', data=df, palette=['#66b3ff', '#ff6666'])

# Annotate each bar with count and percentage
for i, count in enumerate(fraud_counts):
    percent = fraud_percentages[i]
    ax.text(i, count + 100, f"{count} ({percent}%)", ha='center', va='bottom', fontsize=10)

plt.title('Fraud vs Non-Fraud Claim Counts with Percentages')
plt.xlabel('Fraud Reported (0=No, 1=Yes)')
plt.ylabel('Count')
plt.tight_layout()
plt.show()
plt.close()

# Display fraud counts and imbalance ratio
display(Markdown("### Fraud Class Breakdown"))
display(fraud_counts)
imbalance_ratio = fraud_counts.max() / fraud_counts.min()
display(Markdown(f"**Class imbalance ratio (majority/minority):** {imbalance_ratio:.2f}"))

## 4. Convert Fraud Labels for Numeric Analysis

The `fraud_reported` column is already numeric (`0` for non-fraud, `1` for fraud).  
To ensure compatibility with modeling workflows and visual consistency, I duplicate it to a new column `fraud_numeric`.

In [ ]:
# Duplicate the 'fraud_reported' column to a new 'fraud_numeric' column for clarity in modeling workflows
df['fraud_numeric'] = df['fraud_reported']

# Display the first 5 rows of all three columns to confirm the transformation
display(df[['fraud_reported', 'fraud_numeric', 'fraud_label']].head())

## 5. Numeric Feature Exploration

In this section, I explore key **numerical features** to better understand:

- Their **distributions** across all claims (normality, skewness, shape)
- Their **relationship to fraud** using boxplots by fraud label
- The presence of **outliers** that may signal unusual or fraudulent activity

These insights are essential for:
- Identifying features that may help differentiate fraud vs. non-fraud claims
- Detecting values that require **transformation or scaling** before modeling
- Understanding **data quality issues**, like impossible values (e.g., invalid hours)

### Key Features Analyzed:
- `total_claim_amount` – amount claimed by policyholder  
- `incident_hour_of_the_day` – time of day when the incident occurred  
- `risk_score` – engineered fraud risk signal created during ETL

---
> NOTE:
> High-value claims and incidents occurring late at night are often considered red flags in insurance fraud.  
> This numeric feature exploration helps us **quantify how such risk related variables behave**,  
> and whether they demonstrate **distinct patterns between fraudulent and non-fraudulent claims**.
---

In [ ]:
# List of important numeric features to analyze
numeric_features = ['total_claim_amount', 'incident_hour_of_the_day', 'risk_score']

# Check for impossible or invalid hour values (should be between 0 and 23)
invalid_hours = df[(df['incident_hour_of_the_day'] < 0) | (df['incident_hour_of_the_day'] > 23)]
display(Markdown(f"**Invalid 'incident_hour_of_the_day' entries detected:** {len(invalid_hours)}"))

# If invalid entries exist, display a sample for debugging
if len(invalid_hours) > 0:
    display(Markdown("### Sample invalid 'incident_hour_of_the_day' rows"))
    display(invalid_hours.head())

# Generate descriptive statistics (mean, std, min/max, quartiles) for numeric features
display(Markdown("### Descriptive Statistics"))
display(df[numeric_features].describe().T)

# Visualize distribution and fraud-based separation for each numeric feature
for feature in numeric_features:
    fig, axs = plt.subplots(1, 2, figsize=(12, 4))

    # Plot histogram with KDE to visualize overall distribution
    sns.histplot(df[feature], bins=30, kde=True, color='skyblue', ax=axs[0])
    axs[0].set_title(f'Distribution of {feature}')
    axs[0].set_xlabel(feature)
    axs[0].set_ylabel('Number of Claims')
    axs[0].grid(True)

    # Plot boxplot split by fraud class to detect shifts or differences in distributions
    sns.boxplot(x='fraud_numeric', y=feature, data=df, palette=['#66b3ff', '#ff6666'], ax=axs[1])
    axs[1].set_title(f'{feature} by Fraud Status')
    axs[1].set_xlabel('Fraud Reported (0 = No, 1 = Yes)')
    axs[1].set_ylabel(feature)

    # Set overall plot title
    plt.suptitle(f'{feature}: Distribution & Fraud Comparison', fontsize=14)
    plt.tight_layout(rect=[0, 0, 1, 0.95])  # Reserve space for title
    plt.show()
    plt.close()
    
# Correlation Heatmap for Numeric Features
display(Markdown("### Correlation Heatmap of Numeric Features"))

# Compute correlation matrix for numeric features
corr_matrix = df[numeric_features].corr()

# Plot heatmap
plt.figure(figsize=(6, 5))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5)
plt.title('Correlation Heatmap of Numeric Features')
plt.tight_layout()
plt.show()
plt.close()

# Display top 5 largest claims with relevant details for context
top_5_claims = df.nlargest(5, 'total_claim_amount')

# Select columns of interest for context, adjust as per your dataset
cols_to_show = ['claim_number', 'total_claim_amount', 'fraud_reported', 'incident_type', 'collision_type']

# Check if all columns exist, otherwise adjust cols_to_show
existing_cols = [col for col in cols_to_show if col in df.columns]

# Display the top 5 claims with selected columns
display(Markdown("### Top 5 Largest Claims with Details"))
display(top_5_claims[existing_cols])

## 6. Categorical Feature Analysis: Fraud Relationships

This section investigates how key **categorical features** relate to fraud, using both visual and statistical analysis.

### Goals:
- Explore how categories (e.g., collision type) distribute across fraud and non-fraud cases
- Identify categories with **higher fraud rates**
- Visualize **fraud proportions** using stacked bar charts
- Examine whether **missing categorical values** signal higher fraud likelihood

These features are domain-relevant and often cited in insurance fraud studies:
- `incident_type`: Type of incident may influence likelihood of fraudulent reporting.
- `collision_type`: Specific collisions may be staged or prone to fraud.
- `police_report_available`: Lack of police involvement may indicate suspicious behavior.

Key components of this section:
- **Countplots** showing category distribution across fraud vs. non-fraud
- **Fraud rate tables** sorted by likelihood across categories
- **Stacked bar charts** for visualizing proportion of fraud in each category
- **Missing value flags** analyzed for hidden fraud signals

In [ ]:
# Define relevant categorical features for fraud analysis
categorical_features = ['incident_type', 'collision_type', 'police_report_available']

# 1. Plot frequency of each category split by fraud label
plt.figure(figsize=(18, 5))
for i, feature in enumerate(categorical_features, 1):
    plt.subplot(1, 3, i)
    sns.countplot(x=feature, hue='fraud_reported', data=df, palette=['#66b3ff', '#ff6666'])
    plt.title(f'{feature} by Fraud Reported')
    plt.xlabel(feature)
    plt.ylabel('Count')
    plt.xticks(rotation=45)
plt.tight_layout()
plt.show()
plt.close()

# 2. Show fraud rate by category (sorted high to low)
for feature in categorical_features:
    summary = df.groupby(feature).agg(
        count=(feature, 'size'),
        fraud_rate=('fraud_numeric', 'mean')
    ).sort_values('fraud_rate', ascending=False)
    
    display(Markdown(f"### Fraud Rate Summary for `{feature}`"))
    display(Markdown("_Sorted descending by fraud rate to highlight categories with highest fraud risk._"))
    display(summary)

# 3. Visualize fraud vs. non-fraud proportions per category
for feature in categorical_features:
    crosstab = pd.crosstab(df[feature], df['fraud_reported'], normalize='index')
    crosstab.plot(kind='bar', stacked=True, figsize=(8, 4), colormap='Paired')
    plt.title(f'Fraud Proportions by {feature}')
    plt.xlabel(feature)
    plt.ylabel('Proportion of Claims')
    plt.legend(title='Fraud Reported')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
    plt.close()

# 4. Check how missing value flags relate to fraud rates
display(Markdown("### Impact of Missing Values on Fraud Rate"))

# List of expected missing value indicator columns
missing_flags = ['collision_type_missing_flag', 'police_report_available_missing_flag']

for flag_col in missing_flags:
    if flag_col in df.columns:
        # Group by missing flag: count cases and calculate fraud rate
        summary = df.groupby(flag_col).agg(
            count=('fraud_reported', 'size'),
            fraud_rate=('fraud_reported', 'mean')
        ).sort_index(ascending=False)  # Show True (missing) first

        display(Markdown(f"#### Fraud Rate by `{flag_col}`"))
        display(summary)
    else:
        display(Markdown(f"`{flag_col}` not found in dataset. Make sure it was created during ETL."))

## 7. Time Based Trends

Analyzing fraud trends over time may reveal seasonal spikes, evolving fraud tactics, or reporting delays. This insight supports operational decisions, such as increasing fraud monitoring during certain periods.

In [ ]:
if 'incident_date' in df.columns:
    # Convert incident_date to datetime (handle errors by coercing invalid formats to NaT)
    df['incident_date'] = pd.to_datetime(df['incident_date'], errors='coerce')
    
    # Count how many rows will be dropped due to invalid date
    num_invalid_dates = df['incident_date'].isna().sum()
    display(Markdown(f"**Rows with invalid 'incident_date' removed:** {num_invalid_dates}"))

    # Drop rows where incident_date could not be parsed
    time_df = df.dropna(subset=['incident_date'])
    
    # Group by date and fraud status to count claims per day
    fraud_over_time = time_df.groupby(['incident_date', 'fraud_reported']).size().unstack(fill_value=0)
    
    # Add a note about the aggregation granularity and noise
    display(Markdown(
        "**Note:** The plot shows daily claim counts, which can be noisy. " 
        "Consider aggregating by week or month or applying smoothing for clearer trends."
    ))
    
    # Plot fraud and non-fraud claims over time
    plt.figure(figsize=(12, 6))
    fraud_over_time.plot(ax=plt.gca())
    plt.title('Fraud vs Non-Fraud Claims Over Time')
    plt.xlabel('Incident Date')
    plt.ylabel('Number of Claims')
    plt.tight_layout()
    plt.show()
    plt.close()
else:
    display(Markdown("**No 'incident_date' column available to analyze time trends.**"))

# EDA Summary

- Checked Data Quality and Missing Values  
- Explored Fraud Class Imbalance  
- Analyzed Numeric and Categorical Feature Distributions  
- Examined Feature Relationships with Fraud  
- Investigated Missing Data Impact  
- Visualized Time Trends and Outliers

## Next Steps
- Feature Engineering and Selection  
- Train/Test Split and Model Development  
- Model Evaluation and Tuning

---

## Transition to Modeling

The fraud prediction pipeline continues in [`model_training.ipynb`](./model_training.ipynb), where we:
- Engineer new fraud-predictive features  
- Split data into train/test sets  
- Train baseline and advanced models  
- Evaluate fraud detection performance